# Geospatial Analysis with GeoPandas

In [46]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [72]:
try:
    # Intento de cargar el archivo con codificación 'latin1' (común en archivos en español)
    df = pd.read_csv(r'D:/DIPLOMATURA_CIENCIA DE DATOS_PUCP/Python Intermedio/IPRESS.csv', dtype={'UBIGEO': str}, encoding='latin1')
    print("El archivo se cargó correctamente con codificación 'latin1'.")
    
except UnicodeDecodeError:
    # Si 'latin1' falla, el siguiente intento común es con el delimitador incorrecto (e.g., ';' en lugar de ',')
    # Aunque la cabecera parece tener comas, probaremos con el separador más común en estos casos.
    try:
        df = pd.read_csv("IPRESS.csv", dtype={'UBIGEO': str}, sep=';', encoding='latin1')
        print("El archivo se cargó correctamente usando ';' como separador y codificación 'latin1'.")
    except Exception as e:
        # Si ambos fallan, mostramos el error original.
        print(f"Error: No se pudo cargar el archivo. Mensaje de error detallado: {e}")
        # En un escenario real, aquí podrías pedir al usuario el separador o la codificación, pero en este contexto simulamos el intento de arreglo más probable.
        # Para continuar con el plan, usaremos el DataFrame cargado exitosamente en el primer 'try' o en el segundo, si se produjo uno.
        raise # Detener la ejecución si no se pudo cargar el archivo

El archivo se cargó correctamente con codificación 'latin1'.


Renombrar columnas para facilitar el manejo (NORTE -> LATITUD, ESTE -> LONGITUD)

In [33]:

df.rename(columns={'NORTE': 'LATITUD', 'ESTE': 'LONGITUD'}, inplace=True)

 1. Filtrar registros que están "EN FUNCIONAMIENTO" (el "functioning status" es la columna 'Condición' según la inspección del archivo)

In [34]:

df_operacional = df[df['Condición'] == 'EN FUNCIONAMIENTO'].copy()

2. Filtrar registros con coordenadas válidas (LATITUD y LONGITUD no nulas)

Convertir las columnas de coordenadas a tipo numérico, forzando errores a NaN

In [35]:

df_operacional['LATITUD'] = pd.to_numeric(df_operacional['LATITUD'], errors='coerce')
df_operacional['LONGITUD'] = pd.to_numeric(df_operacional['LONGITUD'], errors='coerce')

Eliminar filas donde LATITUD o LONGITUD son NaN después de la conversión

In [36]:

df_filtrado_coords = df_operacional.dropna(subset=['LATITUD', 'LONGITUD'])

3. Seleccionar las columnas clave solicitadas

In [37]:

columnas_clave = [
    'Institución',
    'Nombre del establecimiento',
    'Clasificación',
    'Departamento',
    'Provincia',
    'Distrito',
    'UBIGEO',
    'Estado',
    'LATITUD',
    'LONGITUD'
]

df_final = df_filtrado_coords[columnas_clave]

# Mostrar las primeras filas del DataFrame resultante
print("DataFrame resultante (primeras 5 filas):")
print(df_final.head())

# Mostrar información sobre el DataFrame resultante
print("\nInformación del DataFrame resultante:")
print(df_final.info())

print(f"\nNúmero total de registros originales: {len(df)}")
print(f"Número de registros después de filtrar por 'EN FUNCIONAMIENTO' y coordenadas válidas: {len(df_final)}")

DataFrame resultante (primeras 5 filas):
          Institución         Nombre del establecimiento  \
1   GOBIERNO REGIONAL                             AMBATO   
2   GOBIERNO REGIONAL          SANTA ISABEL DE YUMBATURO   
6               MINSA  PUESTO DE SALUD HEROES DEL CENEPA   
12  GOBIERNO REGIONAL                      NUEVA BETANIA   
15  GOBIERNO REGIONAL                         PONGO ISLA   

                         Clasificación Departamento         Provincia  \
1   PUESTOS DE SALUD O POSTAS DE SALUD    CAJAMARCA           CUTERVO   
2   PUESTOS DE SALUD O POSTAS DE SALUD       LORETO            LORETO   
6   PUESTOS DE SALUD O POSTAS DE SALUD         LIMA              LIMA   
12  PUESTOS DE SALUD O POSTAS DE SALUD      UCAYALI  CORONEL PORTILLO   
15  PUESTOS DE SALUD O POSTAS DE SALUD   SAN MARTIN        SAN MARTIN   

             Distrito  UBIGEO    Estado    LATITUD   LONGITUD  
1          SANTA CRUZ  060611  ACTIVADO -78.858380  -6.133523  
2            PARINARI  160302  

In [39]:
print("DataFrame resultante (primeras 5 filas):")
print(df_final.head())

DataFrame resultante (primeras 5 filas):
          Institución         Nombre del establecimiento  \
1   GOBIERNO REGIONAL                             AMBATO   
2   GOBIERNO REGIONAL          SANTA ISABEL DE YUMBATURO   
6               MINSA  PUESTO DE SALUD HEROES DEL CENEPA   
12  GOBIERNO REGIONAL                      NUEVA BETANIA   
15  GOBIERNO REGIONAL                         PONGO ISLA   

                         Clasificación Departamento         Provincia  \
1   PUESTOS DE SALUD O POSTAS DE SALUD    CAJAMARCA           CUTERVO   
2   PUESTOS DE SALUD O POSTAS DE SALUD       LORETO            LORETO   
6   PUESTOS DE SALUD O POSTAS DE SALUD         LIMA              LIMA   
12  PUESTOS DE SALUD O POSTAS DE SALUD      UCAYALI  CORONEL PORTILLO   
15  PUESTOS DE SALUD O POSTAS DE SALUD   SAN MARTIN        SAN MARTIN   

             Distrito  UBIGEO    Estado    LATITUD   LONGITUD  
1          SANTA CRUZ  060611  ACTIVADO -78.858380  -6.133523  
2            PARINARI  160302  

In [40]:
df_final

,Institución,Nombre del establecimiento,Clasificación,Departamento,Provincia,Distrito,UBIGEO,Estado,LATITUD,LONGITUD
1,GOBIERNO REGIONAL,AMBATO,PUESTOS DE SALUD O POSTAS DE SALUD,CAJAMARCA,CUTERVO,SANTA CRUZ,060611,ACTIVADO,-78.858380,-6.133523
2,GOBIERNO REGIONAL,SANTA ISABEL DE YUMBATURO,PUESTOS DE SALUD O POSTAS DE SALUD,LORETO,LORETO,PARINARI,160302,ACTIVADO,-74.258139,-4.581509
6,MINSA,PUESTO DE SALUD HEROES DEL CENEPA,PUESTOS DE SALUD O POSTAS DE SALUD,LIMA,LIMA,VILLA EL SALVADOR,150142,ACTIVADO,-76.930608,-12.248699
12,GOBIERNO REGIONAL,NUEVA BETANIA,PUESTOS DE SALUD O POSTAS DE SALUD,UCAYALI,CORONEL PORTILLO,CALLERIA,250101,ACTIVADO,-74.296531,-8.398366
15,GOBIERNO REGIONAL,PONGO ISLA,PUESTOS DE SALUD O POSTAS DE SALUD,SAN MARTIN,SAN MARTIN,HUIMBAYOC,220907,ACTIVADO,-75.885812,-6.438298
...,...,...,...,...,...,...,...,...,...,...
20814,GOBIERNO REGIONAL,P.S NUEVO HUALAPAMPA,PUESTOS DE SALUD O POSTAS DE SALUD,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.437187,-5.789407
20815,GOBIERNO REGIONAL,P.S SAN MARTIN DE CONGOÑA,PUESTOS DE SALUD O POSTAS DE SALUD,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.627953,-5.644592
20816,GOBIERNO REGIONAL,P.S TUNAS,PUESTOS DE SALUD O POSTAS DE SALUD,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.546778,-5.498508
20817,GOBIERNO REGIONAL,C.S LIMON DE PORCUYA,CENTROS DE SALUD O CENTROS MEDICOS,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.537903,-5.889830


Abrir archivo geojson

In [89]:
# Upload shape file at centros poblados

distritos = gpd.read_file(r'D:/DIPLOMATURA_CIENCIA DE DATOS_PUCP/Python Intermedio/v_distritos_2023.shp')
distritos


,gid,ubigeo,nombdep,nombprov,nombdist,capital,region_nat,tipo_norma,numero,fecha_fin,comentario,geometry
0,1,030220,APURIMAC,ANDAHUAYLAS,JOSE MARIA ARGUEDAS,HUANCABAMBA,SIERRA,Ley,30295,2014-12-27,None,"POLYGON ((-73.37071 -13.70846, -73.36971 -13.7..."
1,2,030415,APURIMAC,AYMARAES,TINTAY,TINTAY,SIERRA,Ley,13787,1961-12-27,None,"POLYGON ((-73.17529 -13.80103, -73.17388 -13.8..."
2,3,030409,APURIMAC,AYMARAES,LUCRE,LUCRE,SIERRA,Ley,13411,1960-03-24,None,"POLYGON ((-73.24364 -13.81554, -73.24288 -13.8..."
3,4,030214,APURIMAC,ANDAHUAYLAS,SAN MIGUEL DE CHACCRAMPA,CHACCRAMPA,SIERRA,Ley,25235,1990-06-08,None,"POLYGON ((-73.63057 -13.90924, -73.62949 -13.9..."
4,5,030206,APURIMAC,ANDAHUAYLAS,HUAYANA,HUAYANA,SIERRA,Ley,23977,1984-10-30,None,"POLYGON ((-73.51096 -13.91122, -73.51029 -13.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1886,1887,080918,CUSCO,LA CONVENCION,UNION ASHANINKA,MANTARO,SELVA ALTA,Ley,31197,2021-05-18,None,"POLYGON ((-73.79562 -12.26228, -73.79553 -12.2..."
1887,1888,180107,MOQUEGUA,MARISCAL NIETO,SAN ANTONIO,SAN ANTONIO,COSTA,Ley,31216,2021-06-15,None,"POLYGON ((-70.93606 -17.20964, -70.93647 -17.2..."
1888,1889,080910,CUSCO,LA CONVENCION,PICHARI,PICHARI,SELVA ALTA,Ley,26521,1995-08-07,None,"POLYGON ((-73.76176 -12.25292, -73.76175 -12.2..."
1889,1890,050110,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,SAN JUAN BAUTISTA,SIERRA,Ley,13415,1960-04-07,None,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1..."


In [90]:
type(distritos.geometry)

geopandas.geoseries.GeoSeries

In [91]:
distritos['ubigeo'].is_unique

True

In [92]:
distritos['ubigeo'].unique().size

1891

In [93]:
# Select only relevant columns
distritos = distritos[['ubigeo', 'geometry']]
distritos = distritos.rename({'ubigeo':'UBIGEO'}, axis =1 )
distritos

,UBIGEO,geometry
0,030220,"POLYGON ((-73.37071 -13.70846, -73.36971 -13.7..."
1,030415,"POLYGON ((-73.17529 -13.80103, -73.17388 -13.8..."
2,030409,"POLYGON ((-73.24364 -13.81554, -73.24288 -13.8..."
3,030214,"POLYGON ((-73.63057 -13.90924, -73.62949 -13.9..."
4,030206,"POLYGON ((-73.51096 -13.91122, -73.51029 -13.9..."
...,...,...
1886,080918,"POLYGON ((-73.79562 -12.26228, -73.79553 -12.2..."
1887,180107,"POLYGON ((-70.93606 -17.20964, -70.93647 -17.2..."
1888,080910,"POLYGON ((-73.76176 -12.25292, -73.76175 -12.2..."
1889,050110,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1..."


In [94]:
distritos.dtypes

UBIGEO        object
geometry    geometry
dtype: object

In [95]:
# Object or srting to int

distritos['UBIGEO'] = distritos['UBIGEO'].astype(str).astype(int)
distritos

,UBIGEO,geometry
0,30220,"POLYGON ((-73.37071 -13.70846, -73.36971 -13.7..."
1,30415,"POLYGON ((-73.17529 -13.80103, -73.17388 -13.8..."
2,30409,"POLYGON ((-73.24364 -13.81554, -73.24288 -13.8..."
3,30214,"POLYGON ((-73.63057 -13.90924, -73.62949 -13.9..."
4,30206,"POLYGON ((-73.51096 -13.91122, -73.51029 -13.9..."
...,...,...
1886,80918,"POLYGON ((-73.79562 -12.26228, -73.79553 -12.2..."
1887,180107,"POLYGON ((-70.93606 -17.20964, -70.93647 -17.2..."
1888,80910,"POLYGON ((-73.76176 -12.25292, -73.76175 -12.2..."
1889,50110,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1..."


In [96]:
# Ensure the dataset is in WGS-84 (EPSG:4326)
distritos = distritos.to_crs(epsg=4326)
distritos.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [98]:
print("Tipos de dato en 'distritos':")
print(distritos.dtypes['UBIGEO'])

print("\nTipos de dato en 'df_final':")
print(df_final.dtypes['UBIGEO'])

Tipos de dato en 'distritos':
int64

Tipos de dato en 'df_final':
object


In [99]:
# Convertir 'UBIGEO' a string en el primer DataFrame
distritos['UBIGEO'] = distritos['UBIGEO'].astype(str).str.zfill(6)

# Convertir 'UBIGEO' a string en el segundo DataFrame
df_final['UBIGEO'] = df_final['UBIGEO'].astype(str).str.zfill(6)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_21568\123409873.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['UBIGEO'] = df_final['UBIGEO'].astype(str).str.zfill(6)


# Ahora, ejecuta la fusión nuevamente

In [ ]:

dataset_hosp = pd.merge(distritos, df_final, how="inner", on="UBIGEO")

print("¡Fusión realizada con éxito!")
print(dataset_hosp.head())

¡Fusión realizada con éxito!
   UBIGEO                                           geometry  \
0  030220  POLYGON ((-73.37071 -13.70846, -73.36971 -13.7...   
1  030220  POLYGON ((-73.37071 -13.70846, -73.36971 -13.7...   
2  030415  POLYGON ((-73.17529 -13.80103, -73.17388 -13.8...   
3  030415  POLYGON ((-73.17529 -13.80103, -73.17388 -13.8...   
4  030415  POLYGON ((-73.17529 -13.80103, -73.17388 -13.8...   

         Institución Nombre del establecimiento  \
0  GOBIERNO REGIONAL                   SACCLAYA   
1  GOBIERNO REGIONAL                HUANCABAMBA   
2  GOBIERNO REGIONAL              HUANCARPUQUIO   
3  GOBIERNO REGIONAL                  SAN MATEO   
4  GOBIERNO REGIONAL                 TAQUEBAMBA   

                                 Clasificación Departamento    Provincia  \
0           PUESTOS DE SALUD O POSTAS DE SALUD     APURIMAC  ANDAHUAYLAS   
1  CENTROS DE SALUD CON CAMAS DE INTERNAMIENTO     APURIMAC  ANDAHUAYLAS   
2           PUESTOS DE SALUD O POSTAS DE SALUD     

In [101]:
dataset_hosp

,UBIGEO,geometry,Institución,Nombre del establecimiento,Clasificación,Departamento,Provincia,Distrito,Estado,LATITUD,LONGITUD
0,030220,"POLYGON ((-73.37071 -13.70846, -73.36971 -13.7...",GOBIERNO REGIONAL,SACCLAYA,PUESTOS DE SALUD O POSTAS DE SALUD,APURIMAC,ANDAHUAYLAS,JOSE MARIA ARGUEDAS,ACTIVADO,-73.369790,-13.767518
1,030220,"POLYGON ((-73.37071 -13.70846, -73.36971 -13.7...",GOBIERNO REGIONAL,HUANCABAMBA,CENTROS DE SALUD CON CAMAS DE INTERNAMIENTO,APURIMAC,ANDAHUAYLAS,JOSE MARIA ARGUEDAS,ACTIVADO,-73.348327,-13.735002
2,030415,"POLYGON ((-73.17529 -13.80103, -73.17388 -13.8...",GOBIERNO REGIONAL,HUANCARPUQUIO,PUESTOS DE SALUD O POSTAS DE SALUD,APURIMAC,AYMARAES,TINTAY,ACTIVADO,-73.151358,-13.901425
3,030415,"POLYGON ((-73.17529 -13.80103, -73.17388 -13.8...",GOBIERNO REGIONAL,SAN MATEO,PUESTOS DE SALUD O POSTAS DE SALUD,APURIMAC,AYMARAES,TINTAY,ACTIVADO,-73.134188,-13.906555
4,030415,"POLYGON ((-73.17529 -13.80103, -73.17388 -13.8...",GOBIERNO REGIONAL,TAQUEBAMBA,PUESTOS DE SALUD O POSTAS DE SALUD,APURIMAC,AYMARAES,TINTAY,ACTIVADO,-73.099483,-13.896778
...,...,...,...,...,...,...,...,...,...,...,...
7937,050110,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1...",GOBIERNO REGIONAL,SAN JUAN BAUTISTA,CENTROS DE SALUD CON CAMAS DE INTERNAMIENTO,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,ACTIVADO,-74.208830,-13.174033
7938,050110,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1...",GOBIERNO REGIONAL,ÑAHUINPUQUIO,PUESTOS DE SALUD O POSTAS DE SALUD,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,ACTIVADO,-74.205875,-13.181728
7939,050110,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1...",GOBIERNO REGIONAL,LOS OLIVOS,CENTROS DE SALUD O CENTROS MEDICOS,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,ACTIVADO,-74.212919,-13.178064
7940,050110,"POLYGON ((-74.19642 -13.19735, -74.19642 -13.1...",ESSALUD,HOSPITAL II HUAMANGA,HOSPITALES O CLINICAS DE ATENCION GENERAL,AYACUCHO,HUAMANGA,SAN JUAN BAUTISTA,ACTIVADO,-74.200145,-13.175182


# Upload shape file at centros poblados

In [ ]:


ccpp = gpd.read_file(r'D:/DIPLOMATURA_CIENCIA DE DATOS_PUCP/Python Intermedio/CCPP_IGN100K.shp')
ccpp

,OBJECTID,NOM_POBLAD,FUENTE,CÓDIGO,CAT_POBLAD,DIST,PROV,DEP,CÓD_INT,CATEGORIA,X,Y,N_BUSQDA,geometry
0,1,PANDISHARI,INEI,2502010002,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.06462,-10.37129,PANDISHARI,POINT (-74.06462 -10.37129)
1,2,CHICOSA,INEI,2502010003,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.06153,-10.37852,CHICOSA,POINT (-74.06153 -10.37852)
2,3,RAYA,IGN,2502010004,OTROS,RAYMONDI,ATALAYA,UCAYALI,2350,Centro Poblado Menor,-72.94118,-10.33043,RAYA,POINT (-72.94118 -10.33043)
3,4,PENSILVANIA,INEI,2502010005,OTROS,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.05988,-10.40401,PENSILVANIA,POINT (-74.05988 -10.40401)
4,5,PONTE VEDRA,INEI,2502010006,CASERÍO,RAYMONDI,ATALAYA,UCAYALI,2050,Centro Poblado Menor,-74.03788,-10.41809,PONTE VEDRA,POINT (-74.03787 -10.41809)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136582,136584,IPAN,INEI,None,None,ZORRITOS,CONTRALMIRANTE VILLAR,TUMBES,None,None,-81.02462,-81.02462,None,POINT (-81.02462 -4.09728)
136583,136585,URBAN,INEI,None,None,ZORRITOS,CONTRALMIRANTE VILLAR,TUMBES,None,None,-80.84055,-80.84055,None,POINT (-80.84055 -4.06801)
136584,136586,PAJONAL MAJONTONI,IGN,None,None,RAYMONDI,ATALAYA,UCAYALI,2049,Centro Poblado Menor,-74.35804,-74.35804,PAJONAL MAJONTONI,POINT (-74.35804 -10.7274)
136585,136587,AGUA BLANCA,INEI,2501020043,CASERÍO,CAMPOVERDE,CORONEL PORTILLO,UCAYALI,1953,Centro Poblado Menor,-74.78089,-74.78089,AGUA BLANCA,POINT (-74.78089 -8.60511)


In [87]:
# Ensure the dataset is in WGS-84 (EPSG:4326)
ccpp = ccpp.to_crs(epsg=4326)
ccpp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich